In [126]:
# opent the file
f= open('dna_sequence.fasta', "r")

In [186]:
# take a look of the file
#for line in f:
    #print(line)

In [131]:
# aggregate into a dictionary called seqs
f.seek(0)
seqs={}
for l in f:
    l=l.rstrip()
    if l[0]==">":
        words=l.split()
        #print(words)
        name=words[0][1:]
        seqs[name]=''
    else :
        seqs[name]=seqs[name]+l

In [133]:
# the number of records
len(seqs)

18

In [134]:
# max value and min value of the records and the index
values = []
for value in seqs.values():
    values.append(len(value))
print(values)
print("Max value is:", max(values), values.index(max(values)))
print("Min value is:", min(values), values.index(min(values)))

ma_po=values.index(max(values))
mi_po=values.index(min(values))    

[4635, 1151, 4894, 3511, 4076, 2867, 442, 890, 967, 4338, 1352, 4564, 4804, 964, 2095, 1432, 115, 2646]
Max value is: 4894 2
Min value is: 115 16


In [136]:
# identifier of the max and min
key_list = list(seqs.keys()) 

ma_id = key_list[ma_po]
mi_id = key_list[mi_po]
print("id of max seq:", ma_id)
print("id of min seq:", mi_id)

id of max seq: gi|142022655|gb|EQ086233.1|255
id of min seq: gi|142022655|gb|EQ086233.1|346


In [142]:
# 3. use biopython to find the longest open reading frame
from Bio import Seq
import regex as re

In [151]:
# for known identifier, find the longest ORF
startP = re.compile('ATG')
nuc = seqs.get("gi|142022655|gb|EQ086233.1|16")
longest = (0,)
for m in startP.finditer(nuc, overlapped=True):
    if len(Seq.Seq(nuc)[m.start():].translate(to_stop=True)) > longest[0]:
        pro = Seq.Seq(nuc)[m.start():].translate(to_stop=True)
        longest = (len(pro), 
                   m.start(), 
                   str(pro),
                   len(nuc[m.start():m.start()+len(pro)*3+3]))

In [152]:
longest

(547,
 1439,
 'MRAILHRRLSFHPGRHRVMPALTFSWSESDESRSESRRSRSRRGAGRQPVQRRSRADDAGAAHVEVVSLRGAVGRDGDEHRVVHARGRADPGRHVAVAGGDDGAARQPDRARADAADRPCGREARDSVRGARARVVRHAGGEAAGAAARDRRVRLVRDPDLARRQRDLYAAEHPDRQRAAWRRAAGHRHRVRAARMLPRVLGAAALLHLAWHRFDPLARKLVGADQGRDVRGAGVVGNVEGGRLRHDAVGAVAVCRRRQESRAVLGDLLAGADRDGRLLGDARAEHPRLHALRAFAARPGDRPVDRAAVADGAAVGGVGRRDVGDRRDLRQRDLGSDRPDEPDDGHRRGHRARDPHARHDVLQPRRESRRPGVRLLEPVAEGDLVPHRRDDHRDARDRDDAVEDPRDDGRLHLHLARRLLGAARARGGDPDGRLLPDSRHAARHARAVRRARRLQLRARLEPGRAGRARGRRAAEPARLPAHGVSGVVSERAGVLQHALHVRVVRRPRAGVMRVRHLDEVARRTARADRERLIRHPTVNEEATPWQS',
 1644)

In [148]:
# find the longest ORF from all the sequences
n=0
startP = re.compile('ATG')
longest = (0,)
#list_longs=[]
for key, value in seqs.items():
    n=n+1
    for m in startP.finditer(value, overlapped=True):
        if len(Seq.Seq(value)[m.start():].translate(to_stop=True)) > longest[0]:
            pro = Seq.Seq(value)[m.start():].translate(to_stop=True)
            longest = (len(pro), 
                       m.start(), 
                       str(pro),
                       len(value[m.start():m.start()+len(pro)*3+3]),
                       len(value),
                        key)
            #list_longs.append(longest)
print(longest)


(797, 384, 'MEKQSRVTRDGRGRVLCGHRCRGRDWTGHDVRSFRPDSSALLERRHQWDHGSAHHGGDDADGAEPARDGRVRNQRTACVGRVARDARHGARGGRNAAAGMSRQSGWRMRMPATHRRRLAGRRASHSPFAIRHSPFAIRHSPFAIRHSPFAERSIDDGGGHAPEFDMPASSDTANRRASSIARIQRGLKRGLGRRCRFMAAVARGGGMAGPDPEPDRIHRGQEHEREDRPGERAADQRVRERAPEHRMRERNEREHGRHRGKNHRARPLHGRFDDRVERIESRVFVLVDLADQDQRVAHQDPRQRDEADQRIDAERLPEQDQRRHHADQAERARHEHHHHRGDRAHLEDDDHERDRQHDREERQHRLRGLAGFLDGAGFLDSIAARQRGDDRLKLLRDRGRHVRRLQAVGDVAAHRERRHAIAPAHDRLFHPHVDLADLRQRDALPVRAGEREVVEPRRVEPRIAARARDHLYGPDILAYRGDGHPGQQELQLLRDRAGRQPDCLQAVLLQRKVQRRRARAPVGIDRAHHRAAVHHTLDVGCDTAQLVRIGARDPVGHRERRIRAEHELRHAHLRFGREAVGDGAPQPRLQRFALVLAVRADHDLRERRIGQLRPHREIEARRALADVRGDDPRFLLLREPRLDLLRGGARLRNRRAVRHLHLDQYLGAIGDREKLLLHDRHAENREHEEGDDRTGDEPFAGDHARQQLPEAVVAGRVVDRIVAAFEFPEVRQHLHAEVRRERDGDDPRCDQRDSDDPEHVARVFAGGGAREAVRHEADRGDQRAGQHRRGRMAPRVG', 2394, 3511, 'gi|142022655|gb|EQ086233.1|45')


In [175]:
#4. the length of the longest ORF appearing in reading frame 2 of any of the sequences
def find_orf_2(sequence):
    # Find all ATG indexs
    start_position = 1
    start_indexs = []
    stop_indexs = []
    for i in range(1, len(sequence), 3):
        if sequence[i:i+3] == "ATG":
            start_indexs.append(i)

    # Find all stop codon indexs
    for i in range(1, len(sequence), 3):
        stops =["TAA", "TGA", "TAG"]
        if sequence[i:i+3] in stops:
            stop_indexs.append(i)

    orf = []
    mark = 0
    for i in range(0,len(start_indexs)):
        for j in range(0, len(stop_indexs)):
            if start_indexs[i] < stop_indexs[j] and start_indexs[i] > mark:
                orf.append(sequence[start_indexs[i]:stop_indexs[j]+3])
                mark = stop_indexs[j]+3
                break
    return orf


In [176]:
n = 1
lengths = []
for key, value in seqs.items():
    orfs = find_orf_2(value)
    for j in orfs:
        lengths.append(len(j))

# orf_lengths = [len(i) for i in orf_2 if i]
print(max(lengths))

1458


In [184]:
# the starting position of the longest ORF in reading frame 3 in any of the sequences, might be wrong
def find_orf_3(sequence):
    # Find all ATG indexs
    start_position = 2
    start_indexs = []
    stop_indexs = []
    for i in range(2, len(sequence), 3):
        if sequence[i:i+3] == "ATG":
            start_indexs.append(i)

    # Find all stop codon indexs
    for i in range(2, len(sequence), 3):
        stops =["TAA", "TGA", "TAG"]
        if sequence[i:i+3] in stops:
            stop_indexs.append(i)

    orf = []
    mark = 0
    start_position = {}
    for i in range(0,len(start_indexs)):
        for j in range(0, len(stop_indexs)):
            if start_indexs[i] < stop_indexs[j] and start_indexs[i] > mark:
                orf.append(sequence[start_indexs[i]:stop_indexs[j]+3])
                start_position[len(sequence[start_indexs[i]:stop_indexs[j]+3])] = start_indexs[i]
                mark = stop_indexs[j]+3
                break
    return start_position


In [ ]:
find_orf_3(sequence)

In [185]:
n = 1
lengths = []
for key, value in seqs.items():
    print("["+str(n)+"]")
    orfs = find_orf_3(value)
    print(orfs)
    n += 1

[1]
{354: 908, 39: 1400, 129: 2270, 588: 2855, 84: 3449, 150: 3542, 99: 3752, 15: 3941, 48: 4082, 30: 4262, 66: 4310, 81: 4391}
[2]
{147: 620, 120: 908, 42: 1097}
[3]
{171: 776, 9: 1178, 285: 1640, 12: 2021, 66: 2609, 201: 3704, 192: 4025}
[4]
{258: 215, 33: 524, 324: 698, 84: 2627}
[5]
{225: 428, 30: 1940, 48: 3494, 84: 2390, 96: 2696, 63: 3119, 180: 3203, 15: 3398, 72: 3419, 171: 3872}
[6]
{99: 245, 249: 1373}
[7]
{189: 89, 42: 311}
[8]
{}
[9]
{213: 65}
[10]
{252: 1016, 9: 1355, 162: 1394, 417: 1697, 711: 2333, 129: 3059}
[11]
{204: 56, 33: 290, 36: 344, 183: 401, 51: 596}
[12]
{540: 221, 123: 1271, 66: 2057, 132: 2168, 93: 2315, 1401: 3095}
[13]
{1317: 110, 1644: 1439, 132: 3104, 60: 3629, 93: 3797, 153: 4148, 90: 4577}
[14]
{132: 347}
[15]
{63: 692, 27: 1202, 219: 1277, 126: 1961}
[16]
{171: 758, 69: 965, 60: 1136, 27: 1358}
[17]
{}
[18]
{33: 20, 357: 176, 1821: 635}


In [171]:
#  get all the repeats of length of n from all the sequences
def get_all_repeats(sequence):
    length = len(sequence)
    repeats = []
    for i in range(length):
        repeats.append(sequence[i:i + 7])
    return repeats

In [172]:
# execute the defined function and put all the repeats in a list
all_six_repeats = []
for key, value in seqs.items():
    repeats_list = get_all_repeats(value)
    for j in repeats_list:
        all_six_repeats.append(j)

In [173]:
#  get the most frequently occurring repeat of length n in all sequences
def most_common(lst):
    return max(set(lst), key=lst.count)

In [174]:
# sequence of the repeats
most_common(all_six_repeats)

'CGCGCCG'

In [170]:
# number of occurance
print(all_six_repeats.count(most_common(all_six_repeats)))

10
